In [232]:
import csv
import sys
import pandas as pd
import string
import math

TRAIN_SIZE = 80.0

# get user input string
# user_input = input("Enter a review: ")

# initialize TRAIN_SIZE (from user)
input_train_size = int(input("Enter the percentage of data to train on (20-80): "))

#input_train_size = sys.argv[1]
if input_train_size < 20 or input_train_size > 80:
    TRAIN_SIZE = 80
else:
   TRAIN_SIZE = input_train_size




data_pre = pd.read_csv("bumble_google_play_reviews.csv")
data = data_pre.dropna(subset=['content']).reset_index(drop=True)
data['review_class'] = data['score'].apply(lambda x: 'positive' if x > 2 else 'negative')


# 80-20 split
test_index = int(len(data) * 0.8)
train_index = int(len(data) * float(TRAIN_SIZE / 100.0))
train_data = data.iloc[:train_index]
test_data = data.iloc[test_index:]

pos_indices = []
neg_indices = []

for x in range(len(train_data)):
    if train_data['review_class'][x] == 'positive':
        pos_indices.append(x)
    if train_data['review_class'][x] == 'negative':
        neg_indices.append(x)


prior_probabilities = {
    'positive': len(train_data[train_data['review_class'] == 'positive']) / len(train_data),
    'negative': len(train_data[train_data['review_class'] == 'negative']) / len(train_data)
}


In [233]:
vocabulary = []
# list of common stop words
stop_words = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will', 'with']


translation_table = str.maketrans('', '', string.punctuation)

for x in range(len(data['content'])):
    content = data['content'][x]
    lowercase_content = content.lower()
    content_no_punct = lowercase_content.translate(translation_table)
    vocabulary.extend(content_no_punct.split())



unique_vocabulary = []
for item in vocabulary:
    if item not in unique_vocabulary and item not in stop_words:
        unique_vocabulary.append(item)

In [234]:
binary_bag = [[0]*len(unique_vocabulary) for _ in range(len(train_data))]

for x in range(len(train_data)):
    for y in range(len(train_data['content'][x].split())):
        words_in_one_sentence = (train_data['content'][x].split()[y]).lower().translate(translation_table)
        if words_in_one_sentence != '' and words_in_one_sentence not in stop_words:   
            index_of_word = unique_vocabulary.index(words_in_one_sentence)
            binary_bag[x][index_of_word] = 1

# sigma(z) = 1 / (1 + e^(-z))

num_docs_including_word = [0]*len(unique_vocabulary)
num_pos_docs_including_word = [0]*len(unique_vocabulary)
num_neg_docs_including_word = [0]*len(unique_vocabulary)

for word in range(len(unique_vocabulary)):
    for document in range(len(binary_bag)):
        if binary_bag[document][word] == 1:
            num_docs_including_word[word] += 1
            if document in pos_indices:
                num_pos_docs_including_word[word] += 1
            if document in neg_indices:
                num_neg_docs_including_word[word] += 1


print(num_docs_including_word)
print(num_pos_docs_including_word)
print(num_neg_docs_including_word)

[27, 123, 3, 37, 1, 108, 8, 14, 4, 56, 21, 32, 20, 29, 35, 17, 38, 44, 18, 10, 6, 54, 32, 1, 10, 9, 30, 16, 25, 12, 20, 15, 37, 24, 3, 1, 5, 4, 5, 21, 20, 14, 23, 3, 12, 36, 4, 2, 8, 3, 1, 1, 1, 7, 12, 1, 6, 1, 71, 22, 6, 5, 14, 2, 4, 8, 22, 4, 1, 6, 1, 7, 8, 2, 23, 7, 13, 9, 12, 9, 17, 4, 3, 3, 8, 6, 1, 13, 27, 10, 2, 3, 5, 1, 1, 11, 2, 2, 25, 1, 2, 1, 7, 18, 1, 6, 25, 14, 3, 9, 10, 1, 1, 2, 3, 5, 2, 1, 1, 3, 17, 9, 11, 1, 11, 7, 7, 3, 3, 1, 1, 1, 12, 7, 15, 3, 4, 7, 22, 1, 13, 5, 9, 1, 6, 8, 1, 42, 7, 2, 21, 42, 40, 7, 3, 7, 14, 5, 4, 1, 3, 14, 3, 14, 23, 3, 14, 3, 7, 12, 3, 1, 6, 12, 6, 11, 16, 1, 1, 4, 3, 3, 2, 1, 23, 9, 5, 4, 2, 13, 9, 14, 11, 2, 13, 1, 4, 11, 2, 24, 10, 14, 10, 13, 1, 24, 6, 1, 13, 2, 1, 1, 9, 7, 1, 3, 1, 7, 3, 1, 4, 2, 2, 1, 1, 20, 2, 5, 1, 8, 1, 1, 2, 4, 1, 4, 13, 2, 3, 1, 4, 1, 4, 19, 7, 2, 2, 2, 4, 2, 1, 38, 5, 3, 5, 2, 2, 4, 1, 4, 1, 8, 3, 2, 6, 6, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 

In [235]:
print(binary_bag[2])

[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [236]:
prob_word_given_pos_class = []
prob_word_given_neg_class = []

count_pos_class = math.fsum(num_pos_docs_including_word)
count_neg_class = math.fsum(num_neg_docs_including_word)


for x in range(len(unique_vocabulary)):
    prob_word_given_pos_class.append(float(num_pos_docs_including_word[x] + 1) / float(count_pos_class + len(unique_vocabulary)))
    prob_word_given_neg_class.append(float(num_neg_docs_including_word[x] + 1) / float(count_neg_class + len(unique_vocabulary)))

In [237]:
test_data = test_data.reset_index(drop=True)

def naive_bayes_classifier(test_data):
    predictions = []
    # number of unique words in the vocabulary
    V = len(unique_vocabulary)
    # smoothing factor
    alpha = 1
    
    for x in range(len(test_data)):
        pos_prob = math.log(prior_probabilities['positive'])
        neg_prob = math.log(prior_probabilities['negative'])
        document_words = test_data['content'][x].split()

        for word in document_words:
            word = word.lower().translate(translation_table)
            if word != '' and word not in stop_words:
                if word in unique_vocabulary:
                    index_of_word = unique_vocabulary.index(word)
                    # Calculate word probability with smoothing
                    word_pos_prob = (num_pos_docs_including_word[index_of_word] + alpha) / (len(pos_indices) + alpha * V)
                    word_neg_prob = (num_neg_docs_including_word[index_of_word] + alpha) / (len(neg_indices) + alpha * V)
                else:
                    # Apply smoothing for words not in the vocabulary
                    word_pos_prob = alpha / (len(pos_indices) + alpha * V)
                    word_neg_prob = alpha / (len(neg_indices) + alpha * V)
                
                pos_prob += math.log(word_pos_prob)
                neg_prob += math.log(word_neg_prob)
                
                # convert to linear space so that we can compare the probabilities (add up to 1)
                pos_prob = 10**pos_prob
                neg_prob = 10**neg_prob
                
        if pos_prob >= neg_prob:
            predictions.append('positive')
        else:
            predictions.append('negative')
    return predictions

In [238]:
# predictor for a particular review
def naive_bayes_individual(review):
    # number of unique words in the vocabulary
    V = len(unique_vocabulary)
    # alpha is the smoothing factor
    alpha = 1
    
    pos_prob = math.log(prior_probabilities['positive'])
    neg_prob = math.log(prior_probabilities['negative'])
    document_words = review.split()

    for word in document_words:
        word = word.lower().translate(translation_table)
        if word != '' and word not in stop_words: 
            if word in unique_vocabulary:
                index_of_word = unique_vocabulary.index(word)
                word_pos_prob = (num_pos_docs_including_word[index_of_word] + alpha) / (len(pos_indices) + V)
                word_neg_prob = (num_neg_docs_including_word[index_of_word] + alpha) / (len(neg_indices) + V)
            else:
                word_pos_prob = 1 / (len(pos_indices) + V)
                word_neg_prob = 1 / (len(neg_indices) + V)
            
            pos_prob += math.log(word_pos_prob)
            neg_prob += math.log(word_neg_prob)
            
            # convert to linear space so that we can compare the probabilities (add up to 1)
            pos_prob = 10**pos_prob
            neg_prob = 10**neg_prob

    if pos_prob >= neg_prob:
        return 'positive'
    else:
        return 'negative'

In [239]:
# extract actual labels
actual_labels = test_data['review_class'].tolist()
# get predictions
predictions = naive_bayes_classifier(test_data)

In [241]:
def calculate_metrics(predictions, actual_labels):
    # Initialize counters
    TP = FP = TN = FN = 0
    
    # Count occurrences
    for pred, actual in zip(predictions, actual_labels):
        if pred == 'positive' and actual == 'positive':
            TP += 1
        elif pred == 'positive' and actual == 'negative':
            FP += 1
        elif pred == 'negative' and actual == 'positive':
            FN += 1
        elif pred == 'negative' and actual == 'negative':
            TN += 1
    
    # Calculate metrics
    sensitivity = TP / (TP + FN) if TP + FN else 0  
    specificity = TN / (TN + FP) if TN + FP else 0
    precision = TP / (TP + FP) if TP + FP else 0
    negative_predictive_value = TN / (TN + FN) if TN + FN else 0
    accuracy = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN else 0
    F_score = (2 * precision * sensitivity) / (precision + sensitivity) if precision + sensitivity else 0

    return {
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN,
        'Sensitivity (Recall)': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'Negative Predictive Value': negative_predictive_value,
        'Accuracy': accuracy,
        'F-score': F_score
    }

metrics = calculate_metrics(predictions, actual_labels)
for metric, value in metrics.items():
    print(f"{metric}: {value}")


TP: 17
TN: 37
FP: 12
FN: 14
Sensitivity (Recall): 0.5483870967741935
Specificity: 0.7551020408163265
Precision: 0.5862068965517241
Negative Predictive Value: 0.7254901960784313
Accuracy: 0.675
F-score: 0.5666666666666665


### Naive Bayes without stop words

In [242]:
import csv
import sys
import pandas as pd
import string
import math

def naive_bayes_train(train_data):
    # get user input string
    # user_input = input("Enter a review: ")

    # initialize TRAIN_SIZE (from user)
    input_train_size = int(input("Enter the percentage of data to train on (20-80): "))

    #input_train_size = sys.argv[1]
    if input_train_size < 20 or input_train_size > 80:
        TRAIN_SIZE = 80
    else:
        TRAIN_SIZE = input_train_size




    data_pre = pd.read_csv("bumble_google_play_reviews.csv")
    data = data_pre.dropna(subset=['content']).reset_index(drop=True)
    data['review_class'] = data['score'].apply(lambda x: 'positive' if x > 2 else 'negative')


    # 80-20 split
    test_index = int(len(data) * 0.8)
    train_index = int(len(data) * float(TRAIN_SIZE / 100.0))
    train_data = data.iloc[:train_index]
    test_data = data.iloc[test_index:]

    pos_indices = []
    neg_indices = []

    for x in range(len(train_data)):
        if train_data['review_class'][x] == 'positive':
            pos_indices.append(x)
        if train_data['review_class'][x] == 'negative':
            neg_indices.append(x)


    prior_probabilities = {
        'positive': len(train_data[train_data['review_class'] == 'positive']) / len(train_data),
        'negative': len(train_data[train_data['review_class'] == 'negative']) / len(train_data)
    }
    
    vocabulary = []
    # list of common stop words
    stop_words = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will', 'with']


    translation_table = str.maketrans('', '', string.punctuation)

    for x in range(len(data['content'])):
        content = data['content'][x]
        lowercase_content = content.lower()
        content_no_punct = lowercase_content.translate(translation_table)
        vocabulary.extend(content_no_punct.split())



    unique_vocabulary = []
    for item in vocabulary:
        if item not in unique_vocabulary and item not in stop_words:
            unique_vocabulary.append(item)

    binary_bag = [[0]*len(unique_vocabulary) for _ in range(len(train_data))]

    for x in range(len(train_data)):
        for y in range(len(train_data['content'][x].split())):
            words_in_one_sentence = (train_data['content'][x].split()[y]).lower().translate(translation_table)
            if words_in_one_sentence != '' and words_in_one_sentence not in stop_words:   
                index_of_word = unique_vocabulary.index(words_in_one_sentence)
                binary_bag[x][index_of_word] = 1

    # sigma(z) = 1 / (1 + e^(-z))

    num_docs_including_word = [0]*len(unique_vocabulary)
    num_pos_docs_including_word = [0]*len(unique_vocabulary)
    num_neg_docs_including_word = [0]*len(unique_vocabulary)

    for word in range(len(unique_vocabulary)):
        for document in range(len(binary_bag)):
            if binary_bag[document][word] == 1:
                num_docs_including_word[word] += 1
                if document in pos_indices:
                    num_pos_docs_including_word[word] += 1
                if document in neg_indices:
                    num_neg_docs_including_word[word] += 1
    
    prob_word_given_pos_class = []
    prob_word_given_neg_class = []

    count_pos_class = math.fsum(num_pos_docs_including_word)
    count_neg_class = math.fsum(num_neg_docs_including_word)


    for x in range(len(unique_vocabulary)):
        prob_word_given_pos_class.append(float(num_pos_docs_including_word[x] + 1) / float(count_pos_class + len(unique_vocabulary)))
        prob_word_given_neg_class.append(float(num_neg_docs_including_word[x] + 1) / float(count_neg_class + len(unique_vocabulary)))
    
    return prob_word_given_pos_class, prob_word_given_neg_class
        
    
    

In [243]:
test_data = test_data.reset_index(drop=True)

def naive_bayes_classifier_no_stop_words(test_data):
    predictions = []
    # number of unique words in the vocabulary
    V = len(unique_vocabulary)
    # smoothing factor
    alpha = 1
    
    for x in range(len(test_data)):
        pos_prob = math.log(prior_probabilities['positive'])
        neg_prob = math.log(prior_probabilities['negative'])
        document_words = test_data['content'][x].split()

        for word in document_words:
            word = word.lower().translate(translation_table)
            if word != '' and word not in stop_words:
                if word in unique_vocabulary:
                    index_of_word = unique_vocabulary.index(word)
                    # Calculate word probability with smoothing
                    word_pos_prob = (num_pos_docs_including_word[index_of_word] + alpha) / (len(pos_indices) + alpha * V)
                    word_neg_prob = (num_neg_docs_including_word[index_of_word] + alpha) / (len(neg_indices) + alpha * V)
                else:
                    # Apply smoothing for words not in the vocabulary
                    word_pos_prob = alpha / (len(pos_indices) + alpha * V)
                    word_neg_prob = alpha / (len(neg_indices) + alpha * V)
                
                pos_prob += math.log(word_pos_prob)
                neg_prob += math.log(word_neg_prob)
                
                # convert to linear space so that we can compare the probabilities (add up to 1)
                pos_prob = 10**pos_prob
                neg_prob = 10**neg_prob
                
        if pos_prob >= neg_prob:
            predictions.append('positive')
        else:
            predictions.append('negative')
    return predictions

# predictor for a particular review
def naive_bayes_individual_no_stop_words(review):
    # number of unique words in the vocabulary
    V = len(unique_vocabulary)
    # alpha is the smoothing factor
    alpha = 1
    
    pos_prob = math.log(prior_probabilities['positive'])
    neg_prob = math.log(prior_probabilities['negative'])
    document_words = review.split()

    for word in document_words:
        word = word.lower().translate(translation_table)
        if word != '' and word not in stop_words: 
            if word in unique_vocabulary:
                index_of_word = unique_vocabulary.index(word)
                word_pos_prob = (num_pos_docs_including_word[index_of_word] + alpha) / (len(pos_indices) + V)
                word_neg_prob = (num_neg_docs_including_word[index_of_word] + alpha) / (len(neg_indices) + V)
            else:
                word_pos_prob = 1 / (len(pos_indices) + V)
                word_neg_prob = 1 / (len(neg_indices) + V)
            
            pos_prob += math.log(word_pos_prob)
            neg_prob += math.log(word_neg_prob)
            
            # convert to linear space so that we can compare the probabilities (add up to 1)
            pos_prob = 10**pos_prob
            neg_prob = 10**neg_prob

    if pos_prob >= neg_prob:
        return 'positive'
    else:
        return 'negative'

In [244]:
# extract actual labels
actual_labels = test_data['review_class'].tolist()
# get predictions
predictions = naive_bayes_classifier_no_stop_words(test_data)

# get metrics
metrics = calculate_metrics(predictions, actual_labels)
for metric, value in metrics.items():
    print(f"{metric}: {value}")

TP: 17
TN: 37
FP: 12
FN: 14
Sensitivity (Recall): 0.5483870967741935
Specificity: 0.7551020408163265
Precision: 0.5862068965517241
Negative Predictive Value: 0.7254901960784313
Accuracy: 0.675
F-score: 0.5666666666666665


### Naive Bayes with stop words

In [227]:
def naive_bayes_train(train_data):
    global unique_vocabulary
    global prior_probabilities
    global translation_table
    global num_pos_docs_including_word
    global num_neg_docs_including_word
    global pos_indices
    global neg_indices
    
    for x in range(len(train_data)):
        if train_data['review_class'][x] == 'positive':
            pos_indices.append(x)
        if train_data['review_class'][x] == 'negative':
            neg_indices.append(x)
    
    prior_probabilities = {
        'positive': len(train_data[train_data['review_class'] == 'positive']) / len(train_data),
        'negative': len(train_data[train_data['review_class'] == 'negative']) / len(train_data)
    }
    
    vocabulary = []

    translation_table = str.maketrans('', '', string.punctuation)

    for x in range(len(data['content'])):
        content = data['content'][x]
        lowercase_content = content.lower()
        content_no_punct = lowercase_content.translate(translation_table)
        vocabulary.extend(content_no_punct.split())



    for item in vocabulary:
        if item not in unique_vocabulary:
            unique_vocabulary.append(item)

    binary_bag = [[0]*len(unique_vocabulary) for _ in range(len(train_data))]

    for x in range(len(train_data)):
        for y in range(len(train_data['content'][x].split())):
            words_in_one_sentence = (train_data['content'][x].split()[y]).lower().translate(translation_table)
            if words_in_one_sentence != '':   
                index_of_word = unique_vocabulary.index(words_in_one_sentence)
                binary_bag[x][index_of_word] = 1

    # sigma(z) = 1 / (1 + e^(-z))

    num_docs_including_word = [0]*len(unique_vocabulary)
    num_pos_docs_including_word = [0]*len(unique_vocabulary)
    num_neg_docs_including_word = [0]*len(unique_vocabulary)

    for word in range(len(unique_vocabulary)):
        for document in range(len(binary_bag)):
            if binary_bag[document][word] == 1:
                num_docs_including_word[word] += 1
                if document in pos_indices:
                    num_pos_docs_including_word[word] += 1
                if document in neg_indices:
                    num_neg_docs_including_word[word] += 1
    
    prob_word_given_pos_class = []
    prob_word_given_neg_class = []

    count_pos_class = math.fsum(num_pos_docs_including_word)
    count_neg_class = math.fsum(num_neg_docs_including_word)


    for x in range(len(unique_vocabulary)):
        prob_word_given_pos_class.append(float(num_pos_docs_including_word[x] + 1) / float(count_pos_class + len(unique_vocabulary)))
        prob_word_given_neg_class.append(float(num_neg_docs_including_word[x] + 1) / float(count_neg_class + len(unique_vocabulary)))        
    
    

In [ ]:
# ROC Curve with both models
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# get probabilities for each review
pos_probs_no_stop_words = []
neg_probs_no_stop_words = []
pos_probs = []
neg_probs = []

for x in range(len(test_data)):
    pos_prob, neg_prob = naive_bayes_individual_no_stop_words(test_data['content'][x])
    pos_probs_no_stop_words.append(pos_prob)
    neg_probs_no_stop_words.append(neg_prob)
    
    pos_prob, neg_prob = naive_bayes_individual(test_data['content'][x])
    pos_probs.append(pos_prob)
    neg_probs.append(neg_prob)

# get actual labels
actual_labels = test_data['review_class'].tolist()

# get fpr, tpr, and thresholds for both models
fpr_no_stop_words, tpr_no_stop_words, thresholds_no_stop_words = roc_curve(actual_labels, pos_probs_no_stop_words, pos_label='positive')
roc_auc_no_stop_words = auc(fpr_no_stop_words, tpr_no_stop_words)

fpr, tpr, thresholds = roc_curve(actual_labels, pos_probs, pos_label='positive')
roc_auc = auc(fpr, tpr)

# plot ROC curve
plt.figure()
plt.plot(fpr_no_stop_words, tpr_no_stop_words, color='darkorange', lw=2, label='ROC curve without stop words (area = %0.2f)' % roc_auc_no_stop_words)
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve with stop words (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()